In [ ]:
# here we write out different versions of existing heatmap functions to implement droop volt-var + volt-watt control

In [ ]:
# changes to: my_heatmapSetup_funcs.py

In [ ]:
# changes to: my_detControlMatExistence_funcs.py

# indicMat is now 6n x 3n, [Fq Fp]'
def assignF(ver,Fp,Fq,indicMat): # algo similar to updateStateSpace
    
    n=int(len(indicMat)/6) # indicMat has 6n rows for all versions
    Hblock1=np.zeros((3*n,3*n))   
    ridx,colidx=np.nonzero(indicMat[0:3*n,0:3*n]) # python indexing goes first to (last-1)          
    for k in range(len(ridx)):
            Hblock1[ridx[k]][colidx[k]] = Fq
    
    Hblock2=np.zeros((3*n,3*n))   

    if ver==1: # PBC       
        ridx,colidx=np.nonzero(indicMat[0:3*n,0:3*n]) 
        for k in range(len(ridx)):
                Hblock2[ridx[k]][colidx[k]] = Fp

        upper=np.concatenate((Hblock1, np.zeros((3*n,3*n))),axis=1)
        lower=np.concatenate((np.zeros((3*n,3*n)),Hblock2),axis=1)
        F=np.concatenate((upper,lower))  # indicMat is 6n x 6n, [Fq 0 ; 0 Fp]
    
    else: # Droop:        
        ridx,colidx=np.nonzero(indicMat[3*n+1:,0:3*n]) 
        for k in range(len(ridx)):
                Hblock2[ridx[k]][colidx[k]] = Fp
        F=np.concatenate(Hblock1,Hblock2,axis=0) # indicMat is now 6n x 3n, [Fq Fp]'

    #print(F)
    print("Size of F=",F.shape)
    return F

def setupStateSpace(ver, n, feeder, node_index_map, depths):
    #initializes state space matrices A and B
    #n = number of nodes in network
    #feeder = initiaized feeder object
    #node_index_map = dictionary of node indices with node names as keys
    R, X = createRXmatrices_3ph(feeder, node_index_map,depths)
    concat_XR=np.concatenate((X, R), axis = 1)
    
    if ver==1: # PBC       
        A = np.identity(3*n)
        B = concat_XR # (6n*3n) matrix
    else: # volt-watt and volt-var
        A = np.identity(6*n)
        concat_XR_halfs = np.concatenate(((-1/2) * R, (1/2) * X), axis = 1)
        B = np.concatenate((concat_XR, concat_XR_halfs))
    
    return A, B

ctrlTypeList=[] # empty list
def set_ctrlParms(list):
    #nonlocal ctrlTypeList
    ctrlTypeList=list
    return 0

def get_ctrlParms():
    nonlocal ctrlTypeList
    return ctrlTypeList

def updateStateSpace(ver,feeder, n, act_locs, perf_nodes,node_index_map):
    #creates (6n*3n) matrix with 1 at (3i+ph)(3j+ph) for volt-watt control, and (3i+3n+ph)(3j+ph) for volt-var control
    #in the above description, ph is the integer representation (a=0, b=1, c=2) of the phase intersection between the actuator and performance nodes
    #if an actuator and performance node have no phases in common, a warning is printed
    #n = number of nodes in network
    #act_locs = list of actuators locations in network (list of strings)
    #perf_nodes = list of performance nodes 
    #node_index_map = dictionary of node indices for indicMat and F matrix
    if ver==1: # PBC
        indicMat = np.zeros((6*n,6*n))
    else: # volt-watt and volt-var
        indicMat = np.zeros((6*n,3*n))
        
    ctrlTypeList=get_ctrlParms()
    
    # act_loc should now have string format of 'PBCbus_XXX', 'VVCbus_XXX', or 'VWCbus_XXX'
    for i in range(len(act_locs)): 
        act = act_locs[i]
        perf = perf_nodes[i]
        if not(ctrlTypeList[i]=='PBC' or ctrlTypeList[i]=='VVC' or ctrlTypeList[i]=='VWC'):
            raise Exception('Actuator node first 3 chars should be PBC, VVC, or VWC')
            
        except ValueError:
            print("Actuator node first 3 char unrecognised for control type")
        act_phases = feeder.busdict[act[7:]].phases # [7:] extracts the YYY bus number from 'XXXbus_YYY'
        perf_phases = feeder.busdict[perf[7:]].phases
        act_index = node_index_map[act[3:]]
        perf_index = node_index_map[perf[3:]]
        
        phase_intrsct = [ph for ph in act_phases if ph in perf_phases]
        if phase_intrsct == []: # disallow configs in which the act and perf node phases are not aligned. Results in kgain=0.0001 and thinks it's feasible
            print('WARNING: act_node ' + act + ' can NOT track perf_node ' + perf + ' --> no common phases')
            phase_loop_check=False
            break # if any actuator of the config has phase mismtach (between act and perf), don't evaluate the config
        else:
            phase_loop_check=True
            for i in range(len(phase_intrsct)):
                if phase_intrsct[i] == 'a':
                    phase_intrsct[i] = 0
                elif phase_intrsct[i] == 'b':
                    phase_intrsct[i] = 1
                elif phase_intrsct[i] == 'c':
                    phase_intrsct[i] = 2

        if ctrlType=='PBC':
            for ph in phase_intrsct:
                indicMat[(act_index*3)+ph][(perf_index*3)+ph] = 1
                indicMat[(act_index*3)+(3*n)+ph][(perf_index*3)+(3*n)+ph] = 1  
        elif ctrlType=='VVC':
            for ph in phase_intrsct:
                indicMat[(act_index*3)+ph][(perf_index*3)+ph] = 1
        elif ctrlType=='VWC': #volt-watt control
            for ph in phase_intrsct:
                indicMat[(act_index*3)+(3*n)+ph][(perf_index*3)+ph] = 1   
            
    return indicMat,phase_loop_check

In [ ]:
# detControlMatExistence(ver
# computeFeas_v1(ver)
# eval_config(ver,
# find_good_colocated(ver,
# runHeatMapProcess(ver,
# placeMaxColocActs_stopAtInfeas(ver,
# place_max_coloc_acts(ver,

# look for all instances of 
node_index_map[act]
node_index_map[perf]
# and change to
node_index_map[act[3:]]
node_index_map[perf[3:]]

In [2]:
ctrlTypeList=[] # empty list
def set_ctrlParms(list):
    nonlocal ctrlTypeList
    ctrlTypeList=list
    return 0

def get_ctrlParms():
    nonlocal ctrlTypeList
    return ctrlTypeList

mylist={'PBC','PBC'}
set_ctrlParms(mylist)
print(get_ctrlParms)

SyntaxError: no binding for nonlocal 'ctrlTypeList' found (<ipython-input-2-8cf8597fbc8f>, line 3)

In [ ]:
# replace all "(ver" with "(parmObj"

In [ ]:
def find_good_colocated(parmObj,feeder, act_locs, node_index_map, substation_name, depths, file_name, Vbase_ll, Sbase, load_data, headerpath, modelpath):
    #almost the same as runheatmap process, but only runs once and shows one heatmap indicating which nodes are good to place a co-located act/perf node
    #return list of all "green" configs and the associated lzn errors
    #act_locs == a list of pre-set colocated act/perf locations --> to evaluate an empty network, pass in act_locs == []
    a = 0
    ff.clear_graph(feeder) # clear any previous modifictions made to graph
    graph = feeder.network
    heatMapNames = [] # collect heat map names as list of strings
    n = len(graph.nodes) #number of nodes in network
    A, B = setupStateSpace(parmObj,feeder,n,node_index_map, depths)
    feas_configs = [] 
    lzn_error_dic = {} #contains maxLznError for each choice of actuator location with node name as key  
    test_nodes = []
    printCurves=False # your choice on whether to print PVcurves
    graphNodes_nosub = remove_subst_nodes(feeder, file_name) # dont consider co-located at substation nodes, node 650 and 651
    
    cand_ctrlType=['VVC'] # all candidate APNPs will be of this type
    foo=parmObj.get_ctrlTypes()
    parmObj.set_ctrlTypes(cand_ctrlType+foo) #append this control type to front of controlTypes list, to be used in updateStateSpace
    
    
    for node in graphNodes_nosub: # try placing act/perf at all nodes of the network
        if node not in act_locs:
            test_nodes.append(node)
    
    for act in act_locs: 
        markActLoc(graph, act)

    for test in test_nodes:
        print('evaluating act and perf colocated at ',[test]) 
        indicMat,phase_loop_check = updateStateSpace(parmObj,feeder, n, [test] + act_locs, [test] + act_locs, node_index_map) # (n,act,perf,dictionary)
        if phase_loop_check:  # disallow configs in which the act and perf node phases are not aligned
            feas, maxError, numfeas = computeFeas_v1(parmObj,feeder, [test] + act_locs, A, B, indicMat,substation_name,[test] + act_locs, depths, node_index_map,Vbase_ll, Sbase, load_data, headerpath, modelpath,printCurves,file_name) # pass in potential actual loc
            lzn_error_dic[test] = maxError
        else:
            feas=False
            maxError=1
            numfeas=0
            
        markFeas(numfeas, test, graph,phase_loop_check)
        if feas:
            feas_dic = {}
            feas_dic['act'] = [test]
            feas_dic['perf'] = [test]
            feas_dic['lznErr'] = [lzn_error_dic[test]]
            feas_dic['numfeas'] = [numfeas]
            feas_configs += [feas_dic]

    heatMapName='heatmap_colocated' + '_' + file_name
    heatMapNames.append(heatMapName)
    nx.nx_pydot.write_dot(graph, 'generated_figs/'+heatMapName)
    render('dot', 'png', 'generated_figs/'+heatMapName)

    return feas_configs, heatMapNames

In [3]:
a=[5,6,7,8,9]
print(a[:3])
print(a[3:])

[5, 6, 7]
[8, 9]


In [ ]:
def runHeatMapProcess(parmObj,feeder, set_acts, set_perfs, all_act_locs, perf_nodes, node_index_map, substation_name, depths, file_name, Vbase_ll, Sbase, load_data, headerpath, modelpath):
    #compute heatmap (assess feas and lzn error on every node of the feeder, color each red/green on diagram)
    #Then for each act-perf node pair, compute heatmap
    #return list of all "green" configs and the associated lzn errors
    #heatmap shows good places to placed act wrt given perf node, NOT good places to place colocated act-perf node
    #all_act_locs and perf_nodes = lists of node names as strings
    a = 0
    graph = feeder.network
    cur_act_locs = set_acts
    cur_perf_nodes = set_perfs
    heatMapNames = [] # collect heat map names as list of strings
    n = len(graph.nodes) #number of nodes in network
    A, B = setupStateSpace(parmObj,feeder,n, node_index_map, depths)
    lzn_error_run_sum = 0
    feas_configs = []
    
    while a < len(all_act_locs): #outer loop, a = number of actuators to place
        for act in cur_act_locs: 
            markActLoc(graph, act)
            
        lzn_error_dic = {} #contains maxLznError for each choice of actuator location with node name as key  
        test_nodes = []
        graphNodes_nosub = remove_subst_nodes(feeder, file_name) # dont consider co-located at substation nodes, node 650 and 651
        
        for node in graphNodes_nosub:
            if node not in cur_act_locs:
                test_nodes.append(node)
        
        for test in test_nodes: #inner loop
            feas=False # default
            # heatmap color indicates good places to place actuator given chosen loc of perf node (not necessarily colocated)          
            print('evaluating actuator node at ', [test] + cur_act_locs,',\n performance node at ', [perf_nodes[a]] + cur_perf_nodes)
            indicMat,phase_loop_check = updateStateSpace(parmObj,feeder, n, [test] + cur_act_locs, [perf_nodes[a]] + cur_perf_nodes, node_index_map)
            if phase_loop_check:  # disallow configs in which the act and perf node phases are not aligned
                feas, maxError, numfeas = computeFeas_v1(parmObj,feeder, [test] + cur_act_locs, A, B, indicMat, substation_name,[perf_nodes[a]] + cur_perf_nodes, depths, node_index_map, Vbase_ll, Sbase, load_data, headerpath, modelpath, False,file_name) # false for printing PV curves
                lzn_error_dic[test] = maxError
            else:
                maxError=1
                numfeas=0
            
            markFeas(numfeas, test, graph,phase_loop_check)
            if feas:
                feas_dic = {}
                feas_dic['act'] = [test] + cur_act_locs
                feas_dic['perf'] = [test] + cur_perf_nodes
                feas_dic['lznErr'] = [lzn_error_dic[test]]
                feas_dic['numfeas']=[numfeas]
                feas_configs += [feas_dic]        
        
        graph.nodes[perf_nodes[a]]['shape'] = 'square'
        # after generate data for heatmap..
        heatMapName = 'NPP_heatmap_step' + str(a) + '_' + file_name
        heatMapNames.append(heatMapName)
        nx.nx_pydot.write_dot(graph, 'generated_figs/'+heatMapName)
        render('dot', 'png', 'generated_figs/'+heatMapName)
        a += 1 # place actuator
        
        if a <= len(all_act_locs): # choose actuator and finalize assoc perf node
            cur_act_locs = all_act_locs[0:a] # populate cur_act_locs with subset of all_act_locs
            cur_perf_nodes = perf_nodes[0:a] 
            lzn_error_run_sum += lzn_error_dic[cur_act_locs[-1]][0]
            print('The total max linearization error after '+ str(a) +' actuators have been placed = '+ str(lzn_error_run_sum))
            
        # end of while loop
    return feas_configs, lzn_error_run_sum, heatMapNames